# Invertir un vector con seq2seq with attention

In [1]:
import numpy  as np 
import matplotlib.pyplot as plt 
import torch
from torch import nn 
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset,Dataset, DataLoader, random_split, IterableDataset
from torch.nn.utils import clip_grad_norm_
from torch.optim.lr_scheduler import ExponentialLR
import gc
import math


In [11]:
#implementacion de un seq2seq con atencion

class EncoderRNN(nn.Module):
    #esto nos va a devolver un output y un hidden state
    #tenemos tambien un init_hidden para inicializar el hidden state
    #nos interesa el tamaño del input y del hidden state (el output tiene tamaño del hidden)
    def __init__(self,d_input,d_hidden):
        super().__init__()
        self.d_input=d_input
        self.d_hidden=d_hidden
        #vamos a crear un embeding propio con una densa antes de la RNN 
        self.relu=nn.ReLU()
        self.dense1=nn.Linear(d_input,d_hidden)
        self.rnn=nn.GRU(d_hidden,d_hidden,batch_first=True) #como no ponemos batch_first=True todo sera de dimension dim: len x hidden size

    def forward(self,x,hidden):
        x=self.relu(self.dense1(x)) # pasamos B x 1 x input y obtenemos B x 1 x hidden_dim
        output,hidden_out=self.rnn(x,hidden) #obtenemos B x 1 x hidden_dim y 1 x B x hidden_dim 
        return output,hidden_out #obtenemos B x 1 x hidden_dim y 1 x B x hidden_dim
    
    def init_hidden(self,batch_size=1):
        return torch.zeros(1,batch_size,self.d_hidden)


In [12]:
#provemos lo anterior
encoder=EncoderRNN(1,15)
test_vector=torch.randn(5,1,1)
output,attn=encoder(test_vector,encoder.init_hidden(5))
print(output.shape,attn.shape)

torch.Size([5, 1, 15]) torch.Size([1, 5, 15])


In [32]:
class Attention(nn.Module):
    #este es el mecanismo de atencion, el dot product basicamente
    def __init__(self):
        super().__init__()


    def forward(self,hidden_state,encoder_outputs):
        #si el encoder_outputs es:  Batch x len_seq x hidden_dim y el hidden state que usamo de Q es 1 x B x Hidden_dim, devolvemos un context que es B x 1 x hidden_dim  y attn_weights B x 1 x len_seq

        #transponemos el hidden state para hacer las multiplicaciones sobre los batches

        hidden_state_transpose=hidden_state.transpose(0,1) # ahora es B x 1 x hidden
        #el objetivo es multiplicarlo por el encoder_outputs que es  Batch x len_seq x hidden_dim, y para ello lo tenemos que transponer
        encoder_outputs_transpose=encoder_outputs.transpose(1,2) # Batch x  hidden_dim x len_seq 
        #aqui calculamos el alignement

        e=torch.bmm(hidden_state_transpose,encoder_outputs_transpose) # B x 1 x len_seq
        att_weights=F.softmax(e,dim=2) # B x 1 x len_seq y suman 1 sobre los elementos en esa dimension
        context=torch.bmm(att_weights,encoder_outputs) # hacemos la multiplicacion para obtener B x 1 x hidden_dim
        return context, att_weights # return  B x 1 x hidden_dim y B x 1 x len_seq 




In [22]:
attn=Attention()
emb_outs=torch.zeros(2,7,15)
emb_outs[:,3:4,:]=1
context,attn=attn(torch.ones(1,2,15),emb_outs)

tensor([[[ 0.,  0.,  0., 15.,  0.,  0.,  0.]],

        [[ 0.,  0.,  0., 15.,  0.,  0.,  0.]]])


In [33]:
class DecoderRNN(nn.Module):
    #este decoder va a tener implementada la atencion
    #empezaremos el decoder con un hidden_state 0 igual que antes
    #el hidden state previo se le va a meter a modo de query al modelo de atencion, que va a recibir tambien los values, que 
    #son los estados hidden del encoder, de modo que lo ejecutamos en el init de aqui, el encoder
    def __init__(self,input_dim,hidden_size):
        super().__init__()
        self.d_input=input_dim
        self.d_output=input_dim
        self.d_hidden=hidden_size
        #un enmbeding para el input y una densa para el output
        self.dense_in=nn.Linear(input_dim,hidden_size)
        self.dense_out1=nn.Linear(hidden_size*2,hidden_size)
        self.dense_out2=nn.Linear(hidden_size,input_dim) # asumienod dim_input = dim_output

        self.rnn=nn.GRU(hidden_size,hidden_size,batch_first=True)#como no ponemos batch_first=True todo sera de dimension dim: len x hidden size

        #el mecanismo de atencion y los hidden states del encoder
        self.attn=Attention()

    def forward(self,prev_input,hidden_state,encoder_outputs):
        #hacer forward de esto es partiendo del hidden state anterior, sacar el nuevo
        #y con ese calcular la atencion y el context vector, y luego juntarlo con el hidden state y meterlo en la dense y obtener el nuedo hidden

        #el input debe de pasar por el embeding
        x=F.relu(self.dense_in(prev_input)) # entra B x 1 x input_dim y sale B x 1 x Hidden_dim, el 1 es porque estamos procesando un tocken solo
        #ahora calculamos el hidden state
        _,new_hidden_state=self.rnn(x,hidden_state) #entran y_(t-1)=B x 1 x Hidden_dim y h_(t-1)= 1 x Batch_size x Hidden_dim y salen y_t = B x 1 x Hidden_dim y h_t = 1 x B x Hidden_dim

        #con este hidden state,calculamos la attencion, se calcula en cada dimension batch independiente
        #si el encoder_outputs es:  Batch xlen_seq x hidden_dim y el hidden state que usamo de Q es 1 x B x Hidden_dim, devolvemos un context que es B x 1 x hidden_dim  y attn_weights B x 1 x len_seq
        context,attn_weigths=self.attn(new_hidden_state,encoder_outputs) # return  B x 1 x hidden_dim y B x 1 x len_seq 
        new_hidden_state_transpose=new_hidden_state.transpose(0,1)
        #con este context y con el hiden state creamos un vector para meter en una dense
        out=F.sigmoid(self.dense_out1(torch.cat((new_hidden_state_transpose,context),dim=2)))# le metemos B x 1 x 2*hidden_dim y sacamos B x 1 x hidden_dim
        out1=self.dense_out2(out) #metemos B x 1 x hidden_dim y sacamos B x 1 x input_dim

        return out1,new_hidden_state, attn_weigths #B x 1 x input_dim , 1 x B x Hidden_dim y B x 1 x len_seq
    
    def init_hidden(self,batch_size=1):
        return torch.zeros(1,batch_size,self.d_hidden)

In [34]:
decoder=DecoderRNN(1,15)

a=torch.zeros(2,7,15)
a[:,[2],:]=1
o,h,atn=decoder(torch.randn(2,1,1),torch.randn(1,2,15),a)

In [38]:
print(h)

tensor([[[ 0.7764,  0.2985,  0.9831, -0.0340, -0.4747,  0.5859,  0.5646,
          -0.3377,  0.0232, -1.2997, -0.1950, -0.4325,  0.4108,  0.7329,
           0.5236],
         [ 0.0895, -0.4322, -0.8604, -0.4994,  0.2019, -0.3698,  0.3595,
           0.5239,  0.1578,  0.3372,  0.0829,  1.3714,  0.4039,  1.0230,
           0.5676]]], grad_fn=<StackBackward0>)
